The total duration of all annotations combined.

In [3]:
import os
import pandas as pd

duration = 0

for root, dir, files in os.walk('../../../../tuar_processed'):
    for file in files:
        if not file.endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(root, file), comment='#')
        df['duration'] = df['stop_time'] - df['start_time']
        duration += df['duration'].sum()


duration / (60 * 60)
        

np.float64(667.2645379166669)

Names of the features that were fed into the final model.

In [1]:
import joblib
import h5py
import numpy as np

importance_path = '../03 - Training/output/exp_03_12_feature_importance.joblib'

top_n_features = 25
feature_file = '../03 - Training/features/features.hdf5'

importance = joblib.load(importance_path)
top_n_features_idx = np.flip(np.argsort(importance['importances_mean']))[:top_n_features]
session = 'aaaaaaju_s005_t000'
channel = 'A1-T3'

with h5py.File(feature_file) as f:
    features = [feat for feat in f[session][channel].keys() if not feat.startswith('labels')]
    feature_names = []

    for feature in features:
        if f[session][channel][feature].ndim == 1:
            feature_names.append(feature)
        else:
            feature_names.extend([f'{feature}_{i}' for i in range(f[session][channel][feature].shape[1])])

print(np.array(feature_names)[top_n_features_idx])

['ptp_amp' 'rms' 'teager_kaiser_energy_9' 'teager_kaiser_energy_11'
 'variance' 'std' 'teager_kaiser_energy_10' 'wavelet_coef_energy_1'
 'teager_kaiser_energy_8' 'teager_kaiser_energy_1'
 'teager_kaiser_energy_7' 'teager_kaiser_energy_0' 'hurst_exp'
 'line_length' 'pow_freq_bands_2' 'skewness' 'pow_freq_bands_1'
 'teager_kaiser_energy_6' 'kurtosis' 'wavelet_coef_energy_2' 'app_entropy'
 'wavelet_coef_energy_0' 'higuchi_fd' 'pow_freq_bands_0' 'svd_entropy']


In [15]:
import h5py

feature_file = '../03 - Training/features/features.hdf5'
session = 'aaaaaaju_s005_t000'
channel = 'A1-T3'

with h5py.File(feature_file) as f:
    feature_file = '../03 - Training/features/features.hdf5'
    features = [feat for feat in f[session][channel].keys() if not feat.startswith('labels')]
    test = f[session][channel]['teager_kaiser_energy'][:]
    print(test.shape)


(1442, 12)


Es wird überprüft, ob bei den Wavelet Koeffizienten erst die der hohen oder der niedrigen Frequenzen zurückgegeben werden.

In [13]:
import numpy as np
from mne_features.univariate import compute_wavelet_coef_energy

sfreq = 250
duration = 1.0
n_times = int(sfreq * duration)
times = np.arange(n_times) / sfreq

freq_target = 6.0 
signal = np.sin(2 * np.pi * freq_target * times)

data = signal[np.newaxis, :]
compute_wavelet_coef_energy(data)

array([3.33145259e-04, 6.49026269e-02, 1.47066048e+00, 2.07589382e+01,
       1.29640282e+02])

Nun eine ähnliche Überprüfung, aber mit der Teager-Kaiser Energie

In [16]:
import numpy as np
from mne_features.univariate import compute_teager_kaiser_energy

sfreq = 250
duration = 1.0
n_times = int(sfreq * duration)
times = np.arange(n_times) / sfreq

sig_delta = np.sin(2 * np.pi * 2.0 * times)
data_delta = sig_delta[np.newaxis, :]

sig_gamma = np.sin(2 * np.pi * 100.0 * times)
data_gamma = sig_gamma[np.newaxis, :]

delta_tke = compute_teager_kaiser_energy(data_delta)
gamma_tke = compute_teager_kaiser_energy(data_gamma)

print(delta_tke)
print(gamma_tke)

[1.75084423e+01 1.54211903e+01 6.50181029e-01 4.30705147e-01
 8.41398279e-03 1.78822633e-02 3.10326175e-03 1.35257980e-02
 9.75963862e-05 5.71750678e-04 2.28100824e-07 1.40419947e-06]
[3.57491523e-03 8.90242868e-03 8.23321915e-03 1.38729902e-02
 1.16534685e-03 3.19349233e-03 2.91880450e-02 6.17986819e-02
 3.39125888e-03 1.30277313e-02 1.80879064e+00 7.34816055e-02]
